# Neural Networks

In [98]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit as sigmoid
from scipy.optimize import minimize, fmin_cg
from scipy import linalg

In [147]:
def add_dummy_ones(X):
    return np.c_[np.ones(len(X)), X]

def print_shape(name, X):
    print("{name}: {X.shape}".format(name=name, X=X))
    
def print_val(name, val):
    print("{name}: {val}".format(name=name, val=val))
    
def _obs_exp(a, b):
    return "\nObserved: {}\nExpected: {}".format(a, b)

def assert_shape(a, x):
    assert a.shape == x, _obs_exp(a.shape, x)
    
def assertClose(a, b, tol):
    assert np.allclose(a.flatten(), b.flatten(), atol=tol), \
        _obs_exp(a, b)

In [184]:
def test_predict_one_vs_all():
    all_theta = np.array([
            [1, -6, 3],
            [-2, 4, -3]
        ])
    X = np.array([
            [1, 7],
            [4, 5],
            [7, 8],
            [1, 4]
        ])
    X = add_dummy_ones(X)
#     print_val('X w/ bias', X)
    ans = predict_one_vs_all(all_theta, X)
    exp_ans = np.array([[1, 2, 2, 1]]).T  # Column vector
    assert np.array_equal(ans, exp_ans), \
        "\nAnswer: {}\nExpected: {}".format(ans, exp_ans)

        
def predict_one_vs_all(all_theta, X):
    """Predict using a one vs all classifier."""
    m, n = X.shape  # # of features
    num_labels = all_theta.shape[0] # # of classes
    assert all_theta.shape[1] == n, \
        "\nTheta_all: {}\nX: {}".format(all_theta.shape, X.shape)
      
    # The sigmoid function was useful for collapsing real-valued numbers
    # into a 0-1 probability for if something is or is not of class 'y'.
    # Now we can just select from the 
    preds = X.dot(all_theta.T)
    assert_shape(preds, (m, num_labels))
#     print_val('Predictions', preds)
#     print_val('Predictions, sigmoid\'d', sigmoid(preds))
    
    maxes = np.argmax(preds, axis=1)  # Find maxes across rows
    # Since we're using classifier index to represent the digit, we need 
    # to offset by one to align our predictions.
    # Example:
    #   theta_all[0] = classifier for digit 1
    #   ...
    #   theta_all[9] = classifier for digit 10
    maxes += 1  # numpy makes this easy
    maxes = maxes[np.newaxis].T  # Convert to column vector
#     print_val('Predictions, max\'d', maxes)
    assert_shape(maxes, (m, 1))
    return maxes


test_predict_one_vs_all()

In [162]:
def test_one_vs_all():
    X = np.array([
        [8.00000,   1.00000,   6.00000],
        [3.00000,   5.00000,   7.00000],
        [4.00000,   9.00000,   2.00000],
        [0.84147,   0.90930,   0.14112],
        [0.54030,  -0.41615,  -0.98999]
    ])
    X = add_dummy_ones(X)
    y = np.array([1, 2, 2, 1, 3])
    num_labels = 3;
    _lambda = 0.1;
    all_theta = one_vs_all(X, y, num_labels, _lambda);
    exp_all_theta = np.array([
      [-0.559478,   0.619220,  -0.550361,  -0.093502],
      [-5.472920,  -0.471565,   1.261046,   0.634767],
      [ 0.068368,  -0.375582,  -1.652262,  -1.410138]
    ])
    assertClose(all_theta, exp_all_theta, tol=3.01e-3)


def one_vs_all(X, y, num_labels, _lambda, iters=None):
    """One-vs-all multi-class classifier."""
    # Hold our collection of parameters for each classifier 
    m, n = X.shape
    
    # Train a classifier for each label
    # Start with zero for theta for all classifiers
    all_theta = np.zeros((num_labels, n))
    for i in range(num_labels): # Digits range from 1-10
        # Select for that label from our y values
        target_y = (y == (i+1)).astype(int)
        # Minimize using Conjugate Gradient
        result = minimize(
            fun=lr_cost_function,  # use our cost function to evaluate
            x0=all_theta[i%num_labels],  # Initial guess for model parameters
            args=(X, target_y, _lambda),  # Passed into fun
            method='CG', # Conjugate Gradient
            jac=lr_gradient,
            options={'maxiter': iters}  # Same limit as Octave assignment
        )  
        all_theta[i%num_labels] = result.x
        
    return all_theta


test_one_vs_all()

In [165]:
def test_lr_cost_function():
    test_theta = np.array([-2, -1, 1, 2])
    test_X = np.array([
       [1, 8, 1, 6],
       [1, 3, 5, 7],
       [1, 4, 9, 2]
    ])
    test_y = np.array([1, 0, 1])
    test_lambda = 3;
    J = lr_cost_function(test_theta, test_X, test_y, test_lambda);
    assertClose(np.array([J]), np.array([7.6832]), 1e-4)
    
def lr_cost_function(theta, X, y, _lambda=0.1):
    m = len(y)
    onez = np.ones(m)  # 1 x m (row vector)
    
    h = sigmoid(theta.dot(X.T))  # 1 x n * n x m => 1 x m; row vector of probabilities
#     print_val('h', h)
    # When y is negative, i.e. 0, this term goes away due to 'y*log(h)'
    y_pos = np.negative(np.log(h).dot(y)) # 1 x m * m x 1 => 1 x 1 (scalar)
#     print_val('y+', y_pos)
    #     print_shape('y+', y_pos)
    # When y is positive, i.e. 1, this term goes away due to 1-y
#     print_val('1-h', onez-h)
#     print_val('1-y', onez-y)
    y_neg = (np.log(onez-h)).dot(onez-y) # 1 x m * m x 1 => 1 x 1 (scalar)
#     print_val('y-', y_neg)
    #     print_shape('y-', y_neg)
    J = (1/m) * (y_pos - y_neg)
#     print_shape('J', J)
    J += (_lambda/(2*m)) * np.square(theta[1:]).sum()
#     print_shape('lambda/2m * theta^2', reg)
    return J
        

test_lr_cost_function()

In [166]:
def test_lr_gradient():
    test_theta = np.array([-2, -1, 1, 2])
    test_X = np.array([
       [1, 8, 1, 6],
       [1, 3, 5, 7],
       [1, 4, 9, 2]
    ])
    test_y = np.array([1, 0, 1])
    test_lambda = 3;
    grad = lr_gradient(test_theta, test_X, test_y, _lambda=test_lambda)
    exp_grad = np.array([0.31722, -0.12768, 2.64812, 4.23787])
    assertClose(grad, exp_grad, 1e-4)

    
def lr_gradient(theta, X, y, _lambda=None, loud=False):
    """Logistic regression gradient."""
    m = len(y)
    assert_shape(y, (m,)) # row vector of truths
    
    # Make predictions
    h = sigmoid(theta.dot(X.T))  # 1 x n * n x m => 1 x m; row vector of probabilities
    assert h.shape == (m,)
    # Gradients (partial derivatives of cost function for theta)
    #   Theta_j = 1/m * sum(h(x)-y)x
    grad = (1/m) * ((h-y).dot(X)) # (1 x m - 1 x m) * m x n => 1 x n
#     print_shape('gradient', grad)
    grad[1:] += (_lambda/m) * theta[1:] # Skip theta_0
    return grad


test_lr_gradient()

In [190]:
# Now, actually try some predictions
# Import the assignment data
import scipy.io as sio
ex3data = sio.loadmat('ex3/ex3data1.mat')
X, y = ex3data['X'], ex3data['y'][:, 0]
# Add a column of 1s for the bias weight (theta 0)
X = add_dummy_ones(X)
num_labels = len(np.unique(y))
# Train classifiers
all_theta = one_vs_all(X, y, num_labels, 0.1, iters=None)
# Make our predictions
predictions = predict_one_vs_all(all_theta, X)
accuracy = np.mean(predictions[:, 0] == y) * 100
print("Our logistic regression classifier")
print("% Accuracy: ", accuracy)

Our logistic regression classifier
% Accuracy:  96.44


In [191]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(C=10.0)
lr_classifier.fit(X, y)
p = lr_classifier.predict(X)
def print_accuracy(a, b):
    print("% Accuracy:", np.mean(a==b) * 100.)
print("Using scikit-learn's Logistic Regression classifier")
print_accuracy(p, y)

Using scikit-learn's Logistic Regression classifier
% Accuracy: 96.48
